# Оценка линейного искажающего оператора в задаче восстановления изображений

In [ ]:
import numpy as np
from skimage import color, data, restoration, img_as_float, io
# import matplotlib.pyplot as plt
from scipy.signal import convolve2d as conv2

# from skimage import color, data, restoration
# from skimage.restoration import uft
# from scipy.signal import fftconvolve, convolve, convolve2d
# from scipy.stats.stats import pearsonr
# from numpy.fft import fftn, ifftn
from skimage.measure import compare_psnr
# from skimage.draw import bezier_curve
# import scipy.ndimage.filters as fi
# from itertools import chain
# from skimage.draw import line_aa
# from math import sin, cos, pi
from functools import partial
from scipy.optimize import minimize

from util import *
from richardson_lucy import *

In [ ]:
astro = color.rgb2gray(data.astronaut())
liftingbody = img_as_float(io.imread('liftingbody.png'))

In [ ]:
#%interactive graphics
#%matplotlib notebook

In [ ]:
sz = 11
psf = np.ones((sz, sz)) / (sz*sz)
astro_noisy = conv2(astro, psf, 'same')
# Add Noise to Image
#astro_noisy += (np.random.poisson(lam=25, size=astro.shape) - 10) / 255.

In [ ]:
taper_psf = gkern2(51, 10)
plt.imshow(taper_psf)
plt.show()

In [ ]:
# Restore Image using Richardson-Lucy algorithm:
iterations = 7
#taperPSF = gkern2(60, 30)
astro_noisy2 = edgetaper(astro_noisy, taper_psf)
deconv = richardson_lucy_matlab(astro_noisy2, psf, iterations=iterations, eps=1e-5)
show_results(astro, astro_noisy, deconv['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY']])

In [ ]:
lifting_blurred = conv2(liftingbody, psf, 'same')
#taper_psf = gkern2(60,30)
lifting_blurred = edgetaper(lifting_blurred, taper_psf)

In [ ]:
# Restore Image using Richardson-Lucy algorithm my:
iterations = 30
deconv = richardson_lucy_matlab(lifting_blurred, psf, iterations=iterations, eps=1e-5, useFFT=False)
show_results(liftingbody, lifting_blurred, deconv['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY']])

## Линейный оператор смаза

In [ ]:
np.round(motion_blur_psf_my(x=3,y=-4.5),3)

In [ ]:
shift = 30
psf = motion_blur_psf_my(shift, pi/3)
lifting_blurred = conv2(liftingbody, psf, 'same')
iterations = 12
deconv = richardson_lucy_matlab(lifting_blurred, psf, iterations=iterations, eps=1e-5, clip=True, dampar=0.004)
show_results(liftingbody, lifting_blurred, deconv['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY']])
correlation_X = pearsonr(liftingbody.ravel('C')[:-1], liftingbody.ravel('C')[1:])
correlation_Y = pearsonr(liftingbody.ravel('F')[:-1], liftingbody.ravel('F')[1:])
print(correlation_X, correlation_Y)

## Неверная psf

In [ ]:
psf_wrong = motion_blur_psf(shift, pi/4)
#iterations = 20
deconv_wrong = richardson_lucy_matlab(lifting_blurred, psf_wrong, iterations=iterations, eps=1e-5)
show_results(liftingbody, lifting_blurred, deconv_wrong['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY'],
                         deconv_wrong['correlationX'],
                         deconv_wrong['correlationY']], 
          legend=['row correlation', 'column correlation', 'row correlateion(wrong psf)', 'column correlation(wrong psf)'])

## Использование параметра dampar

In [ ]:
s_n, S_find = find_noise(liftingbody)
s_n

In [ ]:
np.sum(motion_blur_psf(shift, pi/4))

In [ ]:
psf = motion_blur_psf(shift, pi/4)
x0_ref = [shift * np.cos(pi/4), shift * np.sin(pi/4)]
lifting_blurred = conv2(liftingbody, psf, 'same')
lifting_blurred = edgetaper(lifting_blurred, taper_psf)
iterations = 30
deconv = richardson_lucy_matlab(lifting_blurred, psf, iterations=iterations, eps=1e-5, clip=True, dampar=s_n, useFFT=True)
show_results(liftingbody, lifting_blurred, deconv['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY']])

### dampar и неверная psf

In [ ]:
psf_wrong = motion_blur_psf_my(shift, pi/3)
#iterations = 40
deconv_wrong = richardson_lucy_matlab(lifting_blurred, psf_wrong, iterations=iterations, eps=1e-5, dampar=s_n)
show_results(liftingbody, lifting_blurred, deconv_wrong['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY'],
                         deconv_wrong['correlationX'],
                         deconv_wrong['correlationY']], 
          legend=['row correlation', 'column correlation', 'row correlateion(wrong psf)', 'column correlation(wrong psf)'])

```
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter


w, h = lifting_blurred.shape
x = np.arange(w)
y = np.arange(h)
X, Y = np.meshgrid(x, y)

fig = plt.figure(figsize=(10,10))
ax = fig.gca(projection='3d')
# Plot the surface.
surf = ax.plot_surface(X, Y, F, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

# Customize the z axis.
#ax.set_zlim(0, 1.01)
#ax.zaxis.set_major_locator(LinearLocator(10))
#ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
#fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()
```

## Кепстр
$$K = F^{-1}\{log(1+\left|F\{I\}\right|)\}$$

In [ ]:
b_clip = 3
N,M = lifting_blurred.shape
# ( ifft2 (100* log (1+ abs ( fft2 ( I ) ) ) ) ) ;
K = np.fft.ifftn(100*np.log(1+np.abs(np.fft.fftn(lifting_blurred))))
K_shift = np.fft.fftshift(K)

In [ ]:
#%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter


h, w = K.shape
x = np.arange(w)
y = np.arange(h)
X, Y = np.meshgrid(x, y)
print(X.shape, Y.shape)
fig = plt.figure(figsize=(10,10))
ax = fig.gca(projection='3d')
# Plot the surface.
H = 1
surf = ax.plot_surface(X, Y, np.abs(K_shift), cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
plt.xlabel('x')
plt.ylabel('y')
# Customize the z axis.
#ax.set_zlim(0, H + H/20)
#ax.zaxis.set_major_locator(LinearLocator(10))
#ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
#fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

In [ ]:
nn = np.argmin(np.real(K_shift))
n, m, = K_shift.shape
x0 = [nn // n - n//2, nn % n - m//2]
print(x0)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(np.clip(np.real(K_shift),-1,1))#, vmin=noisy.min(), vmax=noisy.max())
plt.plot(n//2 + x0[0], m//2 + x0[1], 'ro')
plt.title('Кепстр изображения с выделенным минмиумом')
plt.savefig('kepstr.png')
plt.show()

## Уточнение искажающего оператора
$$\varepsilon = ||\tilde{I}\oplus\tilde{h}-I_0|| \to \min_{(x,y)}$$
Пусть $$\tilde{I}\oplus(\tilde{h}+\tilde{dh})=I_0$$
Тогда $$\tilde{I}\oplus\tilde{dh}=I_0-\tilde{I}\oplus\tilde{h}$$
Получим задачу аналогичную исходной($\tilde{I}\oplus h +\eta = I_0$)

```python
img_diff = liftingbody - convolve2d(deconv['image'], psf, mode='same') # Утечка
deconv_psf = richardson_lucy_matlab(img_diff, deconv['image'], iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)
psf_new = deconv_psf['image']
deconv_upd = richardson_lucy_matlab(lifting_blurred, psf_new, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)
show_results(lifting_blurred, deconv['image'], deconv_upd['image'],
             titles=['blurred', 'restored', 'restored with\nnew psf'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY'],
                         deconv_upd['correlationX'],
                         deconv_upd['correlationY']], 
          legend=['row correlation', 'column correlation', 'row correlateion(new psf)', 'column correlation(new psf)'])
```

Такой подход не сработал :( Будем использовать метод Ньютона
## Уточнение искажающего оператора

$$(x,y)^T_{n+1} = (x, y)^T_{n} - H^{-1}(f(x,y))\nabla f(x,y)$$
$$H(f) =$$

In [ ]:
def funcToMinimize(xy, I_blurred, crop=50, *args, **kwargs):
    psf = motion_blur_psf_my(x=xy[0], y=xy[1])
    restored = richardson_lucy_matlab(I_blurred, psf, *args, **kwargs)
    I_restored = restored['image']
    df = convolve2d(I_restored, psf, 'same') - I_blurred
    return np.mean(np.square(df[crop:-crop, crop:-crop]))

In [ ]:
partial(funcToMinimize, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)([10,10])

In [ ]:
cos(pi/4)*30

### Минимизация методом Нелдера-Мида (симплекс-метод)
Method Nelder-Mead uses the Simplex algorithm [1], [2]. This algorithm is robust in many applications. However, if numerical computation of derivative can be trusted, other algorithms using the first and/or second derivatives information might be preferred for their better performance in general.

Method Powell is a modification of Powell’s method [3], [4] which is a conjugate direction method. It performs sequential one-dimensional minimizations along each vector of the directions set (direc field in options and info), which is updated at each iteration of the main minimization loop. The function need not be differentiable, and no derivatives are taken.

Method CG uses a nonlinear conjugate gradient algorithm by Polak and Ribiere, a variant of the Fletcher-Reeves method described in [5] pp. 120-122. Only the first derivatives are used.

Method BFGS uses the quasi-Newton method of Broyden, Fletcher, Goldfarb, and Shanno (BFGS) [5] pp. 136. It uses the first derivatives only. BFGS has proven good performance even for non-smooth optimizations. This method also returns an approximation of the Hessian inverse, stored as hess_inv in the OptimizeResult object.

Method Newton-CG uses a Newton-CG algorithm [5] pp. 168 (also known as the truncated Newton method). It uses a CG method to the compute the search direction. See also TNC method for a box-constrained minimization with a similar algorithm. Suitable for large-scale problems.

Method dogleg uses the dog-leg trust-region algorithm [5] for unconstrained minimization. This algorithm requires the gradient and Hessian; furthermore the Hessian is required to be positive definite.

Method trust-ncg uses the Newton conjugate gradient trust-region algorithm [5] for unconstrained minimization. This algorithm requires the gradient and either the Hessian or a function that computes the product of the Hessian with a given vector. Suitable for large-scale problems.

Method trust-krylov uses the Newton GLTR trust-region algorithm [14], [15] for unconstrained minimization. This algorithm requires the gradient and either the Hessian or a function that computes the product of the Hessian with a given vector. Suitable for large-scale problems. On indefinite problems it requires usually less iterations than the trust-ncg method and is recommended for medium and large-scale problems.

Method trust-exact is a trust-region method for unconstrained minimization in which quadratic subproblems are solved almost exactly [13]. This algorithm requires the gradient and the Hessian (which is not required to be positive definite). It is, in many situations, the Newton method to converge in fewer iteraction and the most recommended for small and medium-size problems.

```
from scipy.optimize import minimize

res = minimize(partial(funcToMinimize, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True),
               x0=x0, method='nelder-mead',
               options={'xtol': 1e-3, 'disp': True})
```

```
from scipy.optimize import minimize

res = minimize(partial(funcToMinimize, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True),
               x0=x0, method='powell',
               options={'xtol': 1e-3, 'disp': True})
```

## Метод Ньютона

In [ ]:
res_tnc = minimize(partial(funcToMinimize, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True),
               x0=x0, method='TNC')
print(res_tnc)

In [ ]:
res_tnc['x']

In [ ]:
deconv_orig = richardson_lucy_matlab(lifting_blurred, psf, iterations=iterations, dampar=s_n, useFFT=False)
psnr_orig = compare_psnr_crop(liftingbody, deconv_orig['image'], 20)

psf_init = motion_blur_psf_my(x=x0[0], y=x0[1])
deconv_init = richardson_lucy_matlab(lifting_blurred, psf_init, iterations=iterations, dampar=s_n, useFFT=False)
psnr_init = compare_psnr_crop(liftingbody, deconv_init['image'], 20)

x_found, y_found = res_tnc['x']
psf_found = motion_blur_psf_my(x=x_found, y=y_found)
deconv_found = richardson_lucy_matlab(lifting_blurred, psf_found, iterations=iterations, dampar=s_n, useFFT=False)
psnr_found = compare_psnr_crop(liftingbody, deconv_found['image'], 20)

show_results(deconv_orig['image'], deconv_init['image'], deconv_found['image'],
             titles=['Restored with true psf\nPSNR={0}\ncoord={1}'.format(psnr_orig, x0_ref), 
                     'With initial approxiamtion\nPSNR={0}\ncoord={1}'.format(psnr_init, x0),
                     'Minimized error\nPSNR={0}\ncoord={1}'.format(psnr_found, res_tnc['x'])])

## Оценка криволинейного оператора смаза, представленного кривой Безье

In [ ]:
sz = 20
deg = 5
points_real = [(deg-1)*sz//deg-1, sz//deg, sz-1, sz-1]
#points_real = [sz, 0, sz, sz]
psf_bezier = bezier_psf2(points_real)
plt.imshow(psf_bezier)
plt.show()

```
import math
def bezier2_psf(points):
    print(points)
    #points = list(map(int, points))
    shape = math.ceil(max(points[1::2]))+1, math.ceil(max(points[::2]))+1
    r, c = bezier_curve(0, 0, *points, weight=1)
    psf_bezier = np.zeros(shape)
    psf_bezier[r,c] = 1
    psf_bezier /= psf_bezier.sum()
    return psf_bezier
```

In [ ]:
lifting_bezier_blurred = convolve2d(liftingbody, psf_bezier, 'same')

lifting_restored = restoration.richardson_lucy(lifting_bezier_blurred, psf_bezier, iterations=iterations)
psnr_py = compare_psnr_crop(liftingbody, lifting_restored)

deconv_bezier = richardson_lucy_matlab(lifting_bezier_blurred, psf_bezier, iterations=iterations, clip=True, useFFT=True, dampar=0.004)
psnr_my = compare_psnr_crop(liftingbody, deconv_bezier['image'])

In [ ]:
show_results(lifting_bezier_blurred, lifting_restored, deconv_bezier['image'],
            titles=['Blurred image', 'Restored with python LR\nPSNR={0}'.format(psnr_py), 'Restored with my LR\nPSNR={0}'.format(psnr_my)])

### Кепстр

In [ ]:
s_n, S_find = find_noise(lifting_bezier_blurred)

In [ ]:
b_clip = 3
N,M = lifting_bezier_blurred.shape
# ( ifft2 (100* log (1+ abs ( fft2 ( I ) ) ) ) ) ;
K = np.fft.ifftn(100*np.log(1+np.abs(np.fft.fftn(lifting_bezier_blurred))))
K_shift = np.fft.fftshift(K)

In [ ]:
nn = np.argmin(np.real(K_shift))
n, m, = K_shift.shape
x0 = np.array([nn // n - n//2, nn % n - m//2])
print(x0)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(np.clip(np.real(K_shift),-1,1))#, vmin=noisy.min(), vmax=noisy.max())
plt.plot(n//2 + x0[0], m//2 + x0[1], 'ro')
plt.title('Кепстр изображения с выделенным минмиумом')
plt.savefig('kepstr.png')
plt.show()

In [ ]:
x0 = np.concatenate((x0/3, x0))
print(x0)

### Минимизация

In [ ]:
def funcToMinimizeCurved(xy, I_blurred, crop=100, *args, **kwargs):
    psf = bezier_psf2(xy, n=100)
    restored = richardson_lucy_matlab(I_blurred, psf, *args, **kwargs)
    I_restored = restored['image']
    df = convolve2d(I_restored, psf, 'same', 'symm') - I_blurred
    return np.mean(np.square(df[crop:-crop, crop:-crop]))
partial(funcToMinimizeCurved, I_blurred=lifting_bezier_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)([0, 10, 10, 10])

```
#Очень долго (600 запусков)
from scipy.optimize import basinhopping
res_basin = basinhopping(partial(funcToMinimizeCurved, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True), x0, T=10, disp=True, stepsize=5)
print(res_basin['x'])
# >> array([ 4.6543229 , 17.92002928,  9.68371906, 18.01057912])
```

In [ ]:
deconv_orig = richardson_lucy_matlab(lifting_bezier_blurred, psf_bezier, iterations=iterations, dampar=s_n, useFFT=False)
psnr_orig = compare_psnr(liftingbody, deconv_orig['image'])

psf_init = bezier_psf2(x0)
deconv_init = richardson_lucy_matlab(lifting_bezier_blurred, psf_init, iterations=iterations, dampar=s_n, useFFT=False)
psnr_init = compare_psnr(liftingbody, deconv_init['image'])

points_found = res_basin['x']
psf_found = bezier_psf2(points_found)
deconv_found = richardson_lucy_matlab(lifting_bezier_blurred, psf_found, iterations=iterations, dampar=s_n, useFFT=False)
psnr_found = compare_psnr(liftingbody, deconv_found['image'])

show_results(deconv_orig['image'], deconv_init['image'], deconv_found['image'],
             titles=['Restored with true psf\nPSNR={0}\ncoord={1}'.format(psnr_orig, points_real), 
                     'With initial approxiamtion\nPSNR={0}\ncoord={1}'.format(psnr_init, x0),
                     'Minimized error\nPSNR={0}\ncoord={1}'.format(psnr_found, res_basin['x'])])

In [ ]:
#x0 = [30, 0, 20, 20]
res_tnc = minimize(partial(funcToMinimizeCurved, I_blurred=lifting_bezier_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True),
                   options={'eps':.5}, x0=x0, method='TNC')
print(res_tnc)

# [(12.0, 10.0), (20.0, 20.0)]
# [(12.0, 10.0), (20.0, 20.0)]
# [(12.00000001, 10.0), (20.0, 20.0)]
# [(12.0, 10.00000001), (20.0, 20.0)]
# [(12.0, 10.0), (20.00000001, 20.0)]
# [(12.0, 10.0), (20.0, 20.00000001)]
# [(12.0, 10.0), (20.0, 20.0)]
# [(12.0, 10.0), (20.0, 20.0)]
# [(12.00000001, 10.0), (20.0, 20.0)]
# [(12.0, 10.00000001), (20.0, 20.0)]
# [(12.0, 10.0), (20.00000001, 20.0)]
# [(12.0, 10.0), (20.0, 20.00000001)]
#      fun: 0.00502600166295737
#      jac: array([0., 0., 0., 0.])
#  message: 'Local minimum reached (|pg| ~= 0)'
#     nfev: 1
#      nit: 0
#   status: 0
#  success: True
#        x: array([12., 10., 20., 20.])
# ```

In [ ]:
res_cg = minimize(partial(funcToMinimizeCurved, I_blurred=lifting_bezier_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True),
                   options={'eps':1}, x0=x0, method='CG')
print(res_cg)



In [ ]:
nn, mm = 20, 20
a = np.ones((nn, mm))
f = partial(funcToMinimizeCurved, I_blurred=lifting_bezier_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)
for i in range(10, nn):
    for j in range(10):
        xx = np.array([i, j, 20, 20])
        a[j, i] = f(xx)
        print(a[j, i])

In [ ]:
t = np.max(a[:10,10:20])

In [ ]:
t

In [ ]:
plt.imshow(a, 'gray', vmax=t)
plt.title('Goal function value for second point coordinates\ndarker is better')
#plt.savefig('optimization_useless3.png')
plt.show()

In [ ]:
nn = np.argmin(a)

In [ ]:
a.shape

In [ ]:
c = nn %a.shape[0]
r = nn //a.shape[0]

In [ ]:
r, c

In [ ]:
points_real = [(deg-1)*sz//deg-1, sz//deg, sz-1, sz-1]
#points_real = [sz, 0, sz, sz]
psf_bezier = bezier_psf2(points_real)

In [ ]:
a

In [ ]:
X, Y = np.meshgrid(range(nn),range(mm))

In [ ]:
%matplotlib notebook

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, a)
plt.show()

### Построим карту. Есть ли смысл искать минимум?
```
nn, mm = 20, 20
a = np.zeros((nn, mm))
f = partial(funcToMinimizeCurved, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)
for i in range(nn):
    for j in range(mm):
        xx = np.array([i, j, 20, 20])
        a[i, j] = f(xx)
        print(a[i, j])

plt.imshow(-a, 'gray')
plt.title('Goal function value for second point coordinates\nbrighter is better')
plt.savefig('optimization_uselesss.png')
plt.show()
```
<img src="optimization_uselesss2.png">

In [ ]:
a = np.array([[2.81545234e-05, 3.88404645e-03, 3.66117155e-05, 9.93330153e-05,
        2.61873339e-05, 2.65250110e-05, 2.51243228e-05, 2.51861138e-05,
        2.70576292e-05, 2.82764536e-05, 3.60668579e-05, 3.61025950e-05,
        3.62052587e-05, 3.72614259e-05, 4.07473281e-05, 5.21705828e-05,
        7.39831936e-05, 5.51867041e-05, 4.68177891e-04, 7.74594144e-05],
       [4.58434581e-03, 2.32830539e-05, 7.38456871e-03, 2.66604206e-05,
        1.27002199e-03, 2.59056636e-05, 2.97119420e-04, 2.07594333e-05,
        1.32255054e-03, 2.25909143e-05, 5.49360226e-04, 2.76740615e-05,
        6.14222171e-04, 3.02631336e-05, 3.85755896e-04, 3.45004883e-05,
        4.02924805e-04, 4.26709217e-05, 3.15555572e-03, 5.79916970e-05],
       [3.50807022e-05, 5.99194910e-03, 3.04264615e-05, 6.74696021e-04,
        2.97366815e-05, 2.99156296e-05, 2.48862590e-05, 2.29049726e-05,
        2.13673820e-05, 2.08488679e-05, 2.58490400e-05, 2.81313308e-05,
        3.00115674e-05, 3.19233290e-05, 3.33441864e-05, 3.50937788e-05,
        4.77546339e-05, 4.03107290e-05, 4.00828117e-04, 5.34082827e-05],
       [1.21985672e-04, 2.49225493e-05, 1.24717071e-03, 2.75753721e-05,
        4.83431878e-03, 2.58166066e-05, 8.96275407e-04, 1.83327681e-05,
        7.94625641e-04, 1.60037143e-05, 7.64640567e-04, 1.98621887e-05,
        7.20420025e-04, 2.23653327e-05, 2.99340180e-04, 2.52622622e-05,
        4.23991835e-04, 3.02212875e-05, 3.63833374e-03, 4.38115729e-05],
       [2.37471047e-05, 8.34412735e-04, 2.73299459e-05, 4.71804430e-03,
        2.92280857e-05, 2.43425082e-03, 2.28530217e-05, 4.59491505e-05,
        1.81809579e-05, 1.54928894e-05, 1.81196021e-05, 1.97261162e-05,
        2.35079617e-05, 2.07733336e-05, 2.44943671e-05, 2.65082967e-05,
        4.12938526e-05, 3.27260308e-05, 2.87132896e-04, 4.14996957e-05],
       [2.40261698e-05, 2.33173576e-05, 3.13423544e-05, 2.31157410e-05,
        2.58481057e-03, 2.21198085e-05, 5.06963568e-03, 1.52541916e-05,
        2.09266195e-03, 1.11136452e-05, 2.15839506e-03, 1.36641092e-05,
        3.01008898e-04, 1.36474526e-05, 8.34984692e-04, 2.10925150e-05,
        4.74149325e-04, 2.30711004e-05, 3.77096311e-03, 2.76639455e-05],
       [2.38885700e-05, 6.44136341e-04, 2.42859990e-05, 6.35800626e-04,
        2.24537314e-05, 4.92685311e-03, 2.43482376e-05, 1.94442573e-03,
        1.54704808e-05, 1.31850802e-05, 1.26338379e-05, 1.20019242e-05,
        1.41788062e-05, 1.75120211e-05, 1.58899991e-05, 1.86278659e-05,
        2.22090769e-05, 2.46334198e-05, 2.04872457e-04, 2.77792471e-05],
       [2.39796521e-05, 2.06825620e-05, 2.17115451e-05, 1.92557521e-05,
        5.24818234e-05, 1.64365421e-05, 2.11244451e-03, 1.51819842e-05,
        4.96349029e-03, 1.01365526e-05, 6.71460564e-04, 7.45113435e-06,
        5.03908512e-04, 9.62807547e-06, 5.88691569e-04, 1.11614093e-05,
        6.34512253e-04, 1.61039329e-05, 3.99002899e-03, 1.96700294e-05],
       [2.52439218e-05, 2.62602937e-03, 2.14384567e-05, 8.81171043e-04,
        1.92712071e-05, 1.26803633e-03, 1.63492578e-05, 4.84150174e-03,
        1.45209701e-05, 4.50342182e-03, 8.60864842e-06, 1.66120432e-05,
        9.81047302e-06, 9.09749705e-06, 1.16275818e-05, 1.09613550e-05,
        1.45204808e-05, 1.68516554e-05, 1.23587864e-04, 1.98238277e-05],
       [2.58689061e-05, 2.13314294e-05, 2.05284612e-05, 1.75766273e-05,
        1.69555267e-05, 1.31725066e-05, 1.76608934e-05, 1.16447376e-05,
        3.76453025e-03, 5.95269917e-06, 1.81293635e-03, 5.93667433e-06,
        1.51665560e-03, 5.95166576e-06, 3.76067516e-04, 6.91681021e-06,
        1.08487200e-03, 9.34774709e-06, 3.91371899e-03, 1.32899309e-05],
       [3.30599678e-05, 4.18323447e-04, 2.38535806e-05, 1.20270239e-03,
        1.80008797e-05, 2.98896210e-03, 1.45941176e-05, 5.32663346e-04,
        9.43152730e-06, 1.18341132e-03, 7.21222038e-06, 3.52495186e-03,
        7.05420818e-06, 8.59815772e-06, 7.31577839e-06, 8.56830314e-06,
        8.46591722e-06, 1.01695551e-05, 7.24891366e-05, 1.41759281e-05],
       [3.29148857e-05, 2.50170279e-05, 2.57089006e-05, 2.01071700e-05,
        1.95639961e-05, 1.54682822e-05, 1.39201163e-05, 9.03823911e-06,
        2.26130359e-05, 6.13905686e-06, 3.51339604e-03, 5.95872558e-06,
        4.20862961e-03, 5.39856255e-06, 2.16625687e-03, 5.49506383e-06,
        9.40796050e-04, 6.42757331e-06, 5.50006425e-03, 7.87868910e-06],
       [3.50726733e-05, 6.74874955e-04, 2.76492594e-05, 5.66682453e-04,
        2.27392142e-05, 2.39572810e-04, 1.61532980e-05, 5.94632328e-04,
        1.17640413e-05, 1.07466062e-03, 7.33865646e-06, 4.34830226e-03,
        7.75749345e-06, 2.71547292e-03, 6.21182578e-06, 1.64186937e-05,
        6.45714232e-06, 7.65874422e-06, 3.66148557e-05, 9.58679076e-06],
       [3.58504523e-05, 2.83221367e-05, 2.87762576e-05, 2.01419571e-05,
        2.10839952e-05, 1.53288725e-05, 1.95570440e-05, 1.14784489e-05,
        9.83724566e-06, 5.94931457e-06, 8.83602414e-06, 5.11435082e-06,
        2.70735721e-03, 4.23341722e-06, 3.93723214e-03, 3.83227089e-06,
        2.60838909e-03, 4.26879241e-06, 5.81082397e-03, 4.96228071e-06],
       [4.14739543e-05, 6.96400221e-04, 3.25007877e-05, 4.46305050e-04,
        2.28273851e-05, 9.96388078e-04, 1.78425871e-05, 5.37411017e-04,
        1.32006403e-05, 2.94381907e-04, 7.15640043e-06, 3.45400155e-03,
        5.95421625e-06, 3.75519691e-03, 5.70832876e-06, 3.02289578e-03,
        7.62837389e-06, 1.35508230e-05, 2.09017248e-05, 8.23960991e-06],
       [5.20616563e-05, 3.54083357e-05, 3.56046993e-05, 2.61613398e-05,
        2.63246590e-05, 2.12346154e-05, 1.94100752e-05, 1.24899589e-05,
        1.15171531e-05, 6.65262193e-06, 8.22782523e-06, 5.38330964e-06,
        1.54988726e-05, 4.03464102e-06, 2.76347528e-03, 3.62622833e-06,
        4.47330241e-03, 3.53178588e-06, 9.58331116e-03, 3.48585527e-06],
       [1.11828645e-04, 1.11596076e-03, 6.86130120e-05, 1.05612979e-03,
        5.91111904e-05, 1.12836836e-03, 2.31971665e-05, 1.15728158e-03,
        1.74616103e-05, 1.21606213e-03, 7.93649828e-06, 1.07400226e-03,
        6.54646633e-06, 1.55386618e-03, 7.68157322e-06, 4.22301079e-03,
        3.95199845e-05, 3.57790294e-03, 2.97431079e-05, 7.94532762e-05],
       [5.41853893e-05, 4.54385614e-05, 4.29340660e-05, 3.25682283e-05,
        3.59799073e-05, 2.67128139e-05, 2.42814216e-05, 1.69067630e-05,
        1.77463443e-05, 8.91652505e-06, 9.76618726e-06, 5.98761607e-06,
        7.92150732e-06, 4.29881705e-06, 1.62299189e-05, 3.45387059e-06,
        3.48441407e-03, 3.19723054e-06, 9.61889961e-03, 3.18845239e-06],
       [3.33879534e-04, 4.06445471e-03, 2.59670855e-04, 4.72744017e-03,
        1.94481460e-04, 4.81396281e-03, 1.50071423e-04, 4.98052897e-03,
        9.52636825e-05, 5.65478678e-03, 6.61300448e-05, 6.65613487e-03,
        3.09442124e-05, 7.62225673e-03, 2.38338365e-05, 1.22933011e-02,
        3.74269304e-05, 1.10354597e-02, 7.24035188e-05, 4.66220283e-03],
       [6.33396903e-05, 5.20363127e-05, 5.21186727e-05, 4.28441145e-05,
        4.17961410e-05, 3.23075286e-05, 3.35183203e-05, 2.46040033e-05,
        2.45131245e-05, 1.48650610e-05, 1.36976661e-05, 7.70540544e-06,
        8.70828586e-06, 4.70557073e-06, 7.64665807e-06, 3.49294183e-06,
        9.07514279e-05, 3.18137826e-06, 4.45300968e-03, 3.16768018e-06]])

In [ ]:
378//19

In [ ]:
points_found = res_tnc['x']
psf_found = bezier_psf2(points_found)
deconv_found = richardson_lucy_matlab(lifting_bezier_blurred, psf_found, iterations=iterations, dampar=s_n, useFFT=False)

In [ ]:
deconv_orig = richardson_lucy_matlab(lifting_bezier_blurred, psf_bezier, iterations=iterations, dampar=s_n, useFFT=False)
psnr_orig = compare_psnr(liftingbody, deconv_orig['image'])

psf_init = bezier_psf2(x0)
deconv_init = richardson_lucy_matlab(lifting_bezier_blurred, psf_init, iterations=iterations, dampar=s_n, useFFT=False)
psnr_init = compare_psnr(liftingbody, deconv_init['image'])

points_found = res_tnc['x']
psf_found = bezier_psf2(points_found)
deconv_found = richardson_lucy_matlab(lifting_bezier_blurred, psf_found, iterations=iterations, dampar=s_n, useFFT=False)
psnr_found = compare_psnr(liftingbody, deconv_found['image'])

show_results(deconv_orig['image'], deconv_init['image'], deconv_found['image'],
             titles=['Restored with true psf\nPSNR={0}\ncoord={1}'.format(psnr_orig, points_real), 
                     'With initial approxiamtion\nPSNR={0}\ncoord={1}'.format(psnr_init, x0),
                     'Minimized error\nPSNR={0}\ncoord={1}'.format(psnr_found, res_tnc['x'])])

In [ ]:
#x0 = [12, 10, 20, 20]
res_nelder = minimize(partial(funcToMinimizeCurved, I_blurred=lifting_bezier_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True), x0=x0, method='Nelder-Mead', options={'eps':1, 'xtol': 1e-3, 'disp': True})
print(res_nelder)

In [ ]:
deconv_orig = richardson_lucy_matlab(lifting_bezier_blurred, psf_bezier, iterations=iterations, dampar=s_n, useFFT=False)
psnr_orig = compare_psnr(liftingbody, deconv_orig['image'])

psf_init = bezier_psf2(x0)
deconv_init = richardson_lucy_matlab(lifting_bezier_blurred, psf_init, iterations=iterations, dampar=s_n, useFFT=False)
psnr_init = compare_psnr(liftingbody, deconv_init['image'])

points_found = res_nelder['x']
psf_found = bezier_psf2(points_found)
deconv_found = richardson_lucy_matlab(lifting_bezier_blurred, psf_found, iterations=iterations, dampar=s_n, useFFT=False)
psnr_found = compare_psnr(liftingbody, deconv_found['image'])

show_results(deconv_orig['image'], deconv_init['image'], deconv_found['image'],
             titles=['Restored with true psf\nPSNR={0}\ncoord={1}'.format(psnr_orig, points_real), 
                     'With initial approxiamtion\nPSNR={0}\ncoord={1}'.format(psnr_init, x0),
                     'Minimized error\nPSNR={0}\ncoord={1}'.format(psnr_found, res_nelder['x'])])

* Запустить градиентный метод
* Сделать мощный edgetaper
* Сделать антиалиасинг psf